1️⃣ Qué es un Servicio de Logging Distribuido

En pocas palabras:

Es un sistema centralizado que recibe registros (logs) de muchos servicios diferentes.

Cada log dice qué pasó, cuándo y dónde, incluyendo errores, advertencias, información o debug.

Permite monitorizar, depurar y auditar sistemas distribuidos, incluso cuando algo falla y nadie se dio cuenta.

Ejemplo: en un sistema de microservicios:

El servicio de pagos genera un error

El servicio de usuarios genera info de sesión

Ambos envían logs a tu servidor central

Vos podés ver todo desde un mismo lugar, filtrarlo y analizarlo

📌 Esto se usa en empresas, apps grandes, servidores web, APIs, sistemas financieros, IoT y prácticamente en cualquier software que no sea un script individual.

2️⃣ Por qué es necesario

Depuración efectiva: encontrar errores rápidamente.

Auditoría y seguridad: saber qué hizo cada servicio, cuándo y por quién.

Escalabilidad: sistemas distribuidos con múltiples servicios necesitan un lugar central para los logs.

Alertas y monitoreo: detectar problemas antes de que exploten.

“Los sistemas caen. Los logs sobreviven.” — DevOps mantra 😎

3️⃣ Qué necesitás saber antes de arrancar

Para este proyecto, conviene conocer:

🔹 Backend / API

Cómo crear un servidor web en Python (Flask es ideal y simple).

Cómo definir endpoints POST y GET (/logs, /stats).

Cómo recibir datos JSON y procesarlos.

🔹 Bases de datos

Guardar y consultar logs: SQLite, PostgreSQL o cualquier DB que conozcas.

Filtrar por fechas, niveles de severidad, servicios, etc.

🔹 Autenticación básica

Entender headers HTTP y cómo verificar tokens.

Responder con errores claros si el token es inválido.

In [ ]:
from flask import Flask, request, jsonify
from datetime import datetime, timezone
import sqlite3

app = Flask(__name__)

RUTA_DB = "logs.db"

# -------------------------
# Inicializar la base de datos
# -------------------------
def inicializar_db():
    conexion = sqlite3.connect(RUTA_DB)
    cursor = conexion.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS logs (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            fecha_evento TEXT,
            servicio TEXT,
            nivel TEXT,
            mensaje TEXT,
            recibido_en TEXT
        )
    """)
    conexion.commit()
    conexion.close()

inicializar_db()

# -------------------------
# Tokens válidos
# -------------------------
TOKENS_VALIDOS = {
    "TOKEN_SERVICIO_A": "servicio-a",
    "TOKEN_SERVICIO_B": "servicio-b",
}

# -------------------------
# Funciones útiles
# -------------------------
def hora_actual_iso():
    return datetime.now(timezone.utc).isoformat()

def verificar_token():
    autorizacion = request.headers.get("Authorization", "")
    if not autorizacion.startswith("Token "):
        return None
    token = autorizacion.split(" ")[1].strip()
    if token in TOKENS_VALIDOS:
        return token
    return None

# -------------------------
# Endpoint para recibir logs
# -------------------------
@app.route("/logs", methods=["POST"])
def recibir_logs():
    token = verificar_token()
    if not token:
        return jsonify({"error": "Quién sos, bro?"}), 401

    datos = request.get_json(silent=True)
    if not datos:
        return jsonify({"error": "JSON inválido"}), 400

    # Permitir lista de logs o un solo log
    lista_logs = datos if isinstance(datos, list) else [datos]

    conexion = sqlite3.connect(RUTA_DB)
    cursor = conexion.cursor()
    for log in lista_logs:
        fecha_evento = log.get("timestamp", hora_actual_iso())
        servicio = log.get("service", TOKENS_VALIDOS[token])
        nivel = log.get("severity", "INFO")
        mensaje = log.get("message", "")
        recibido_en = hora_actual_iso()

        cursor.execute("""
            INSERT INTO logs (fecha_evento, servicio, nivel, mensaje, recibido_en)
            VALUES (?, ?, ?, ?, ?)
        """, (fecha_evento, servicio, nivel, mensaje, recibido_en))
    conexion.commit()
    conexion.close()

    return jsonify({"ok": True, "guardados": len(lista_logs)}), 201

# -------------------------
# Endpoint para consultar logs
# -------------------------
@app.route("/logs", methods=["GET"])
def consultar_logs():
    conexion = sqlite3.connect(RUTA_DB)
    cursor = conexion.cursor()
    cursor.execute("SELECT * FROM logs ORDER BY id DESC")
    filas = cursor.fetchall()
    conexion.close()

    logs = []
    for fila in filas:
        logs.append({
            "id": fila[0],
            "fecha_evento": fila[1],
            "servicio": fila[2],
            "nivel": fila[3],
            "mensaje": fila[4],
            "recibido_en": fila[5],
        })
    return jsonify({"cantidad": len(logs), "logs": logs})

# -------------------------
# Ejecutar servidor
# -------------------------
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000, debug=True)
